# Lab 10: Linear Regression and Feature Engineering

Due Date: Friday April 10, 2020 at 11:59PM

In this lab, we will work through the process of:
1. Implementing a linear model 
1. Determining features 
1. Minimizing loss functions 

This lab will continue using the tips dataset.

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** at the top of this notebook.

*List collaborators here*
Madi Perez

In [1]:
# Run this cell

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.optimize import minimize

# Packages for configuration

import sys

In [2]:
# TEST 

assert 'pandas' in sys.modules and "pd" in locals()
assert 'numpy' in sys.modules and "np" in locals()
assert 'matplotlib' in sys.modules and "plt" in locals()
assert 'seaborn' in sys.modules and "sns" in locals()

# Loading the Tips Dataset

To begin, let's load the tips dataset from the `seaborn` library.  This dataset contains records of tips, total bill, and information about the person who paid the bill. This is the same dataset used in Lab 5, so it should look familiar!

In [3]:
data = sns.load_dataset("tips")

print("Number of Records:", len(data))
data.head()

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Question 1: Defining the Model and Feature Engineering

In the previous lab, we defined a constant model with only one parameter. 

Now let's make a more complicated model that utilizes other features in our dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 \cdot \text{total_bill} + \theta_2 \cdot \text{sex} + \theta_3 \cdot \text{smoker} + \theta_4 \cdot \text{day} + \theta_5 \cdot \text{time} + \theta_6 \cdot \text{size}
$$

Notice that some of these features are not quantitative variables! But our linear model will need to predict a number. 

Let's start by converting some of these non-numerical values into numerical values. Below we split the tips and the features.

In [4]:
tips = data['tip']
X = data.drop(columns='tip')

In [5]:
# TEST 

assert X.shape == (244, 6)
assert len(tips) == 244           

## Question 1a: Feature Engineering

First, let's convert our features to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. 

For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentionally assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7, and Monday to the numeric value of 1. In our linear model, Sunday will have 7 times the influence of Monday, which can lower the accuracy of our model.

Instead, let's use one-hot encoding to better represent these features! 

As discussed in lecture, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset, allowing us to see the transformed dataset named `one_hot_X`. This dataframe holds our "featurized" data.

You should use the function `pd.get_dummies`.

In [6]:
X.head()

,total_bill,sex,smoker,day,time,size
0,16.99,Female,No,Sun,Dinner,2
1,10.34,Male,No,Sun,Dinner,3
2,21.01,Male,No,Sun,Dinner,3
3,23.68,Male,No,Sun,Dinner,2
4,24.59,Female,No,Sun,Dinner,4


In [7]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input data.
    
    Parameters
    -----------
    data: a dataframe that may include non-numerical features
    
    Returns
    -----------
    A one-hot encoded dataframe that only contains numeric features
    """
    return pd.get_dummies(data)
    # YOUR CODE HERE
    #raise NotImplementedError()

one_hot_X = one_hot_encode(X)
one_hot_X.head()

,total_bill,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,2,0,1,0,1,0,0,0,1,0,1
1,10.34,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3,1,0,0,1,0,0,0,1,0,1
3,23.68,2,1,0,0,1,0,0,0,1,0,1
4,24.59,4,0,1,0,1,0,0,0,1,0,1


In [8]:
# TEST
assert one_hot_X.shape == (244, 12)


## Question 1b: Defining the Model

Now that all of our data is numeric, we can begin to define our model function. Notice that after one-hot encoding our data, we now have 12 features instead of 6. Therefore, our linear model now looks like:

$$
\text{Tip} = \theta_1 \cdot \text{size} + \theta_2 \cdot \text{total_bill} + \theta_3 \cdot \text{day_Thur} + \theta_4 \cdot \text{day_Fri} + ... + \theta_{11} \cdot \text{time_Lunch} + \theta_{12} \cdot \text{time_Dinner}
$$

We can represent the linear combination above as a matrix-vector product. Implement the `linear_model` function to evaluate this product.

You can use [np.dot](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.dot.html), [pd.DataFrame.dot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dot.html), or the `@` operator to multiply matrices/vectors. However, while the `@` operator can be used to multiply `numpy` arrays, it generally will not work between two `pandas` objects, so keep that in mind when computing matrix-vector products!


In [9]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    
    Parameters
    -----------
    thetas: a 1D vector representing the parameters of our model ([theta1, theta2, ...])
    X: a 2D dataframe of numeric features
    
    Returns
    -----------
    A 1D vector representing the linear combination of thetas and features as defined above.
    """
    return X.dot(thetas)
    # YOUR CODE HERE
    #raise NotImplementedError()

In [10]:
# TEST
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

In [11]:
# TEST
assert (linear_model(2*np.eye(100), np.ones(100)) == 2*np.ones(100)).all()


# Question 2: Fitting the Model to the Data

Recall in the previous lab, we defined the functions 

- Mean Absolute Error (MAE) : average of the absolute loss
- Mean Square Error (MSE) : average of the square loss

These empirical risk functions measure the difference between observed values and predicted values. To implement these functions you can use `numpy` functions `np.mean` and `np.abs`.

# Question 2a

Implement Mean Absolute Error. Note that `y` is the observed response variable and `y_hat` is the predicted response variable.

In [12]:
def mae(y, y_hat):
    return np.mean(np.abs(y - y_hat))
    # YOUR CODE HERE
    #raise NotImplementedError()

In [13]:
# TEST 

assert mae(np.arange(1,10,1), np.arange(-10,-1,1)) == 11

# Question 2b

Implement Mean Square Error. Note that `y` is the observed response variable and `y_hat` is the predicted response variable.

In [14]:
def mse(y, y_hat):
    # YOUR CODE HERE
    return np.mean((y - y_hat)**2)
    #raise NotImplementedError()

In [15]:
# TEST 

assert mse(np.arange(1,10,1), np.arange(-10,-1,1)) == 121

# Question 2c

In Lab 9, we fit the constant model to the data with the `minimize` function in the `scipy` package. Here we need to use `minimize` to fit the linear regression model to the data. Note that `minimize` has the form 

```python 
minimize(..., x0=...)['x']
```
We need to specify two arguments

- the function being minimized 
- the initial guess for the input that minimizes the output. 

We will take the initial guess for each theta value to be 0. We need the following data to specify the function being minimized

- average loss function : an empirical risk function
- model : a linear model 
- X: a table of feature variables
- y: an array of response variables

Remember that the inputs to the function being minimized are the theta values. So we need a function that inputs these four objects and outputs an appropriate function of theta values. 

In [16]:
def make_function_to_minimize(average_loss_function, model, X, y):
    def my_function_to_minimize(theta):
        return average_loss_function(y, model(theta, X))
    
    return my_function_to_minimize

# Question 2ci

Use `make_function_to_minimize` with 

- average loss function : `mse` from Question 2b
- model : `linear_model` from Question 1b
- X: `one_hot_X` from Question 1a
- y: `tip` consisting of the `tips` column of `data`

to determine a function called `mse_function_to_minimize`. We will use it with `minimize` to determine the theta values.


In [17]:
mse_function_to_minimize = make_function_to_minimize(mse, linear_model, one_hot_X, tips)

# YOUR CODE HERE
#raise NotImplementedError()

mse_thetas = minimize(mse_function_to_minimize, x0=np.zeros(one_hot_X.shape[1]))['x']

In [18]:
# TEST 

assert len(mse_thetas) == 12


We have fit the linear model to the data

In [19]:
print("The values for theta that minimize Mean Square Error are: \n\n ", mse_thetas)
print("\n The value of the Mean Square Error is: \n\n", mse(linear_model(mse_thetas, one_hot_X), tips))

The values for theta that minimize Mean Square Error are: 

  [0.094487   0.1759912  0.18411073 0.21655221 0.15712765 0.24353564
 0.0151997  0.17746083 0.05600925 0.15199322 0.23440138 0.16626186]

 The value of the Mean Square Error is: 

 1.010353561236632


# Question 2cii

Use `make_function_to_minimize` with 

- average loss function : `mae` from Question 2a
- model : `linear_model` from Question 1b
- X: `one_hot_X` from Question 1a
- y: `tip` consisting of the `tips` column of `data`

to determine a function called `mae_function_to_minimize`. We will use it with `minimize` to determine the theta values.


In [20]:
mae_function_to_minimize = make_function_to_minimize(mae, linear_model, one_hot_X, tips)

# YOUR CODE HERE
#raise NotImplementedError()

mae_thetas = minimize(mae_function_to_minimize, x0=np.zeros(one_hot_X.shape[1]))['x']

In [21]:
# TEST 

assert len(mse_thetas) == 12


We have fit the linear model to the data.

In [22]:
print("The values for theta that minimize Mean Absolute Error are: \n\n ", mae_thetas)
print("\n The value of the Mean Absolute Error is: \n\n", mae(linear_model(mae_thetas, one_hot_X), tips))

The values for theta that minimize Mean Absolute Error are: 

  [ 0.10219208  0.08580053  0.11824163  0.28118845  0.13522405  0.26420604
 -0.05763028  0.20613719 -0.03544616  0.2863693   0.17768954  0.22174044]

 The value of the Mean Absolute Error is: 

 0.7204095075409332
